# Data Analytics

A hands-on workshop by [Le Wagon](https://www.lewagon.com)

## Agenda

- Python 101
- Your first Exploratory Data Analysis with Pandas, matplotlib and Seaborn

We will code in a [Notebook](https://jupyter.org/)

## Python 101

### Simple Built-in Types

Here are the basic types we need:

- Numeric (`int` and `float`)
- Text (`str`)
- Boolean (`bool`)

Ask yourself: what's the type for a _street number_ , a _zip code_ , an _annual income_ , etc.

### Variables

Storing and re-using values in memory:

```python
city = "Paris"
```

### Functions

Exemple: [built-in functions](https://docs.python.org/3/library/functions.html)

```python
len("Paris")
```

### Objects

Stored in **variables**. We can call **methods** (~functions) _on_ them.

Example:

```python
name = "boris"
name.capitalize()
```

Sometimes we call methods _with arguments_ :

```python
name.endswith("s")
```

## Data Sourcing

The art of _collecting_ data priori to an analysis

### Finding Data

- I can export it from a software (CSV)
- I know it exists somewhere in a database
- It's on this website I visit daily
- I have found a service (API?) that gives access to it
- I received a file by email

### Data is in a file (or many files!)

- File - Structured Data (CSV, JSON, XML or XLSX)
- File - Unstructured Data (PDF, DOCX, TXT or XSLX)
- File - Binary and/or proprietary (How can I open it?)

### Data is online

On the Internet or behind company VPN

- It's on a simple **SQL** database and I have access to it
- It's on Google BigQuery or another Data Warehouse provider (SQL usually works too)
- There is an **API** & Documentation
- There is not. **Web Scraping** is the only option

### Finding a Dataset

- [Google Dataset Search](https://datasetsearch.research.google.com/)
- [Kaggle](https://www.kaggle.com/)

## Pandas

[pandas.pydata.org](https://pandas.pydata.org/)

### Why not Excel?

- Handling **large** datasets (no limit on number of rows)
- Combine multiple files
- Pull data from APIs
- Python!

### Pandas IO Tools

Simplest way of using Pandas is to load data directly **from a CSV** file on disk:

```python
import pandas as pd

a_dataframe = pd.read_csv(FILEPATH)
```

Pandas can also load data from multiple other sources:

👉 [pandas.pydata.org/pandas-docs/stable/user_guide/io.html](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)

### Seaborn

We will use [`seaborn`](https://seaborn.pydata.org) for **Data Visualization** (it's based on `matplotlib`)

```python
import seaborn as sns
```

First, let's choose a dataset to explore

### Dataset of a neuropsychology experiment 

Let's use this Dataset from 2018 used to redact a [master thesis](https://cibleplus.ulb.ac.be/discovery/fulldisplay?context=L&vid=32ULDB_U_INST:32ULB_VU1&search_scope=MyInst_and_CI&tab=Everything&docid=alma991009576612504066) in neuropsychology focused on the resistance to interference from external distractors

### Loading the CSV

We can use the [`pandas.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.

```python
results_df = pd.read_csv('data/experiments_results.csv', sep=';')
```

We now have `results_df`, a variable containing our first [**DataFrame**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

### Quick look

Let's use some attributes & methods of the DataFrame:

- `.shape`
- `.columns`
- `.dtypes`
- `.head()` / `tail()`

### Cleaning Data

We can show how many nulls are found in each column with:

```python
results_df.isnull().sum()
```

We can drop some columns we find not relevant enough for our analysis:

```python
columns_to_drop = ['id', 'academic_lvl']
results_df.drop(columns_to_drop, axis="columns", inplace=True)
```

We can replace `NaN` with:

```python
results_df.fillna({'fatigue_post': results_df['fatigue_pre']}, inplace=True)
```

### Columns & Rows

We can filter **columns** (by name):
    
```python
results_df['age'] # => Series
# or
results_df[['age', 'gender', 'fatigue_pre']] # => DataFrame
```

Or **rows** (by index):

```python
results_df[5:10]
```

And even **combine** both approaches

### Boolean Indexing

```python
results_df['gender'] == 'F'
```

Using this Series as an selection index:

```python
results_df[results_df['gender'] == 'F']
```

## Exploratory Data Analysis

Quoting [Wikipedia](https://en.wikipedia.org/wiki/Exploratory_data_analysis):
    
> In statistics, exploratory data analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods.

Let's start asking some **questions** 🤔 to our dataset

🤔 What are the 5 most tirred participants?

```python
results_df.nlargest(5, 'fatigue_pre')
```

🤔 What are the age groups with results?

```python
results_df['age'].unique()
```

🤔 Follow-up: How many participants per age group?

```python
results_df['age'].value_counts()
```

We can plot a barchart with `matplotlib` calling `.plot(kind='bar')` directly _on_ the Dataframe

```python
results_df['fatigue_pre'].value_counts().head(5).plot(kind='bar')
```

No need for a `value_counts()` with `seaborn`, we can just use a [Countplot](https://seaborn.pydata.org/generated/seaborn.countplot.html)

```python
sns.countplot(data=results_df, x='fatigue_pre')
```

```python
order = results_df['fatigue_pre'].value_counts().index
sns.countplot(data=results_df, x='fatigue_pre', order=order)
```

🤔 What's the influence of the gender on **fatigue_pre**?

```python
sns.countplot(data=results_df, x="fatigue_pre", hue="gender")
```

🤔 What is the **distribution** of the total scores?

Start by creating a selection of relevant columns:

```python
cols = [col for col in results_df.columns if col.startswith('q')]
cols
```

Then create a column summing the questions' answers:

```python
results_df['score'] = results_df[cols].sum(axis=1)
```

We can use a [Distplot](https://seaborn.pydata.org/generated/seaborn.distplot.html):

```python
sns.distplot(results_df['score'])
```

How many participants performed **perfectly (24/24)**? Could we remove those participants from the visualization? What technique can we use?

```python
imperfect_df = results_df[results_df['score'] == 24]
sns.distplot(imperfect_df['score'])
```

We can ask Pandas for the mean:

```python
results_df.price.mean()
imperfect_df.price.mean()
```

🤔 What is the **distribution** of scores based on the gender?

We can use a [Violinplot](https://seaborn.pydata.org/generated/seaborn.violinplot.html):

```python
sns.violinplot(data=imperfect_df, x="gender", y="score")
```

We can also add more features:

```python
sns.violinplot(data=imperfect_df, x="gender", y="score", hue="age")
```

We can make the figure bigger calling this before the `sns.violinplot(...)`

```python
from matplotlib import pyplot as plt

plt.figure(figsize=(15, 8))
```

## Going Further

Some topics for another time

### SQL

You can't be _too_ good at SQL!


```sql
SELECT
    ROUND(EXTRACT(epoch FROM age(_camp_starts_at, birthday)) / (3600 * 24 * 365)) as age,
    COUNT(DISTINCT id)
FROM alumni
GROUP BY 1
HAVING ROUND(EXTRACT(epoch FROM age(_camp_starts_at, birthday)) / (3600 * 24 * 365)) > 15
AND ROUND(EXTRACT(epoch FROM age(_camp_starts_at, birthday)) / (3600 * 24 * 365)) < 90
ORDER BY 1
```

### API Fetching

You need the [`requests`](https://requests.readthedocs.io/en/master/) package.

```python
import requests

url = "https://..."
response = requests.get(url) # `GET` HTTP request
print(response.status_code)  # Should be 200 if OK
data = response.json()

# TODO: Load `data` in a Pandas DataFrame
```

### Web Scraping

You need the [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) package and some front-end development knowledge (DOM, CSS selectors, etc.)

```python
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# You now can query the `soup` object!
soup.title.string
soup.find('h1')
soup.find_all('a')
# etc...
```

### Machine Learning

Do your first regressions, classification & clustering with the [`scikit-learn`](https://scikit-learn.org/stable/) package.


## Your turn!